## Emergency Detection

In [2]:
import pandas as pd
from detection import emergency_detection_workflow, prepare_data_for_detection, detect_emergency
from base.minio_utils import load_model_from_minio
from base.influx_utils import fetch_all_sensor_data
from base.homecare_hub_utils import send_info, send_todo

ModuleNotFoundError: No module named 'detection'

In [ ]:
sensor_data = fetch_all_sensor_data(
    start_hours=24,
    interval_hours=24
)
prepared_df = prepare_data_for_detection(sensor_data)
prepared_df.tail(5)

In [ ]:
room_stats = load_model_from_minio("occupancy")
room_stats

In [ ]:
_, message = emergency_detection_workflow()
print(message)

In [ ]:
room = "livingroom"
duration = 240  # seconds
stats = pd.Series({"mean": 120, "std": 30})

_, result = detect_emergency(room, duration, stats)
print(result)

In [ ]:
send_todo("🚨 Patient Emergency! 🚨", result, 2)

## Motion Analysis

In [ ]:
from motion_analysis import (
    visualize_transitions_heatmap, 
    plot_bidirectional_transaction_graph, 
    summary_of_sleep_time_and_time_outside,
    count_daily_visits,
    detect_wake_up_times,
    detect_went_to_sleep_times,
    create_separate_heat_maps)
from base.minio_utils import load_model_from_minio
import pandas as pd
from base.homecare_hub_utils import send_info
from motion_analysis import analyse_motion_patterns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
old_model = load_model_from_minio("motion", 2)
new_model = load_model_from_minio("motion", 1)

In [ ]:
old_model.head()

In [ ]:
plot_bidirectional_transaction_graph(old_model)

In [ ]:
plot_bidirectional_transaction_graph(new_model)

In [ ]:
visualize_transitions_heatmap(old_model)

#### Counting the sleep and went outside periods

In [ ]:
summary_of_sleep_time_and_time_outside(old_model)

In [ ]:
summary_of_sleep_time_and_time_outside(new_model)

#### Detect daliy trips to places

In [ ]:
count_daily_visits(new_model, transition_state='bathroom')

In [ ]:
count_daily_visits(new_model, transition_state='kitchen')

In [ ]:
count_daily_visits(new_model, transition_state='livingroombedarea')

In [ ]:
count_daily_visits(old_model, transition_state='livingroomdoor')

#### Wake up times

In [ ]:
detect_wake_up_times(new_model)

In [ ]:
detect_went_to_sleep_times(old_model)

In [ ]:
create_separate_heat_maps(new_model)

In [ ]:
message = analyse_motion_patterns()

In [ ]:
print(message)

## Burglary Detection

In [6]:
from burglary_detection import BurglaryDetector
from burglary_detection import create_burglary_message
from motion_model import train_motion_model
import pandas as pd
from base import base_logger

In [2]:
detector = BurglaryDetector(contamination=0.01, model_type='burglary')
detector.load_model(version=1)

Model version 1 loaded from MinIO successfully.


In [4]:
motion_model = train_motion_model(start_hours=24*7*5, interval_hours=24*7*5, time_threshold_seconds=1800)
detection = detector.detect(motion_model)

In [5]:
detection

(                   from                 to              leave_time  \
 0        livingroomdoor            kitchen 2024-11-29 17:43:15.000   
 1               kitchen  livingroombedarea 2024-11-29 18:05:28.848   
 2     livingroombedarea            kitchen 2024-11-29 18:06:06.029   
 3               kitchen       went outside 2024-11-29 18:23:10.914   
 4          went outside  livingroombedarea 2024-11-30 03:45:58.649   
 ...                 ...                ...                     ...   
 1671           bathroom            kitchen 2025-01-03 11:19:50.113   
 1672            kitchen           bathroom 2025-01-03 11:31:46.201   
 1673           bathroom            kitchen 2025-01-03 11:33:54.773   
 1674            kitchen           bathroom 2025-01-03 12:18:42.269   
 1675           bathroom            kitchen 2025-01-03 12:21:32.604   
 
                   enter_time  duration  leave_hour  anomaly anomaly_label  
 0    2024-11-29 17:53:00.438   585.438          17        1        N

In [7]:
create_burglary_message(detection[0], detection[1])

'🚔 **Burglary Alert!** 🚔\nPotential burglary detected with the following details:\n\n🔹 **From:** went outside ➡️ **To:** livingroombedarea\n🔸 **Leave Time:** 2024-11-30 03:45:58\n🔸 **Enter Time:** 2024-11-30 03:45:58\n\n🔹 **From:** livingroombedarea ➡️ **To:** went outside\n🔸 **Leave Time:** 2024-12-03 14:59:02\n🔸 **Enter Time:** 2024-12-03 14:59:02\n\n🔹 **From:** livingroombedarea ➡️ **To:** livingroomdoor\n🔸 **Leave Time:** 2024-12-03 22:05:43\n🔸 **Enter Time:** 2024-12-03 22:24:11\n\n🔹 **From:** livingroomdoor ➡️ **To:** livingroombedarea\n🔸 **Leave Time:** 2024-12-03 22:24:11\n🔸 **Enter Time:** 2024-12-03 22:46:27\n\n🔹 **From:** livingroombedarea ➡️ **To:** livingroomdoor\n🔸 **Leave Time:** 2024-12-03 23:04:56\n🔸 **Enter Time:** 2024-12-03 23:12:12\n\n🔹 **From:** livingroombedarea ➡️ **To:** went to sleep\n🔸 **Leave Time:** 2024-12-04 00:14:02\n🔸 **Enter Time:** 2024-12-04 00:14:02\n\n🔹 **From:** bathroom ➡️ **To:** livingroombedarea\n🔸 **Leave Time:** 2024-12-07 00:48:57\n🔸 **Ente

In [8]:
from burglary_detection import detect_burglary

In [9]:
detect_burglary()

Model version 1 loaded from MinIO successfully.


(False,
 '✅ **All Clear!** No unusual activities detected.\nEverything is normal. Have a great day! 😊')